In [1]:
import pandas as pd
import geopandas as gpd
import re
import requests
import os

In [2]:
# Bangkok district list
bkk_district = [
    "พระนคร","ป้อมปราบศัตรูพ่าย","สัมพันธวงศ์","ปทุมวัน","บางรัก","ยานนาวา","สาทร","บางคอแหลม",
    "ดุสิต","บางซื่อ","พญาไท","ราชเทวี","ห้วยขวาง","ดินแดง","พระโขนง","คลองเตย","ประเวศ",
    "สวนหลวง","บางเขน","ดอนเมือง","จตุจักร","บางกะปิ","ลาดพร้าว","บึงกุ่ม","หนองจอก",
    "มีนบุรี","ลาดกระบัง","ธนบุรี","คลองสาน","บางกอกน้อย","บางพลัด","บางกอกใหญ่","ภาษีเจริญ",
    "บางขุนเทียน","จอมทอง","ตลิ่งชัน","ราษฎร์บูรณะ","หนองแขม","วัฒนา","บางแค","หลักสี่","สายไหม",
    "คันนายาว","สะพานสูง","วังทองหลาง","คลองสามวา","บางนา","ทวีวัฒนา","ทุ่งครุ","บางบอน"
]

# Extract district string from address
def extract_district(address: str):
    if not isinstance(address, str):
        return "ไม่ทราบ"
    # case 1: contains "เขตxxx"
    m = re.search(r"เขต([^\s]+)", address)
    if m:
        return m.group(1)
    # case 2: check district list inside address
    for d in bkk_district:
        if d in address:
            return d
    return "ไม่ทราบ"


In [3]:
# Department Store
url_department = "https://data.bangkok.go.th/dataset/d8f814ac-cbaf-43c3-9576-f533b2554776/resource/438101c3-5535-4fe2-bc5e-83aa73703d4a/download/department_store.csv"
df_department = pd.read_csv(url_department)
print("Columns:", df_department.columns)
print("Examples:")
print(df_department.head())

Columns: Index(['id_depart', 'name', 'address', 'tel', 'time', 'dcode', 'dname', 'lat',
       'lng'],
      dtype='object')
Examples:
   id_depart                                        name  \
0          2  บิ๊กซี สาขาสุขาภิบาล 3 สาขา 2 (เอ็กซ์ตร้า)   
1          3                          โลตัส จรัญสนิทวงศ์   
2          4                             โลตัส แจ้งวัฒนะ   
3          5                           โลตัส ซีคอนสแควร์   
4          6                                 โลตัส บางแค   

                                             address          tel  \
0  643 ถนน รามคำแหง แขวงหัวหมาก เขตบางกะปิ กรุงเท...  0 2735 3062   
1  244 ซอย จรัญสนิทวงศ์ 79 แขวงบางพลัด เขตบางพลัด...  0 2434 7575   
2  300 ม.1 ถ.แจ้งวัฒนะ แขวงทุ่งสองห้อง เขตหลักสี่...  0 2990 7580   
3  904/2 ม.6 ถ.ศรีนครินทร์ แขวงหนองบอน เขตประเวศ ...  0 2721 9118   
4  266 ถ. เพชรเกษม แขวงบางแคเหนือ เขตบางแค กทม. 1...  0 2804 4837   

                      time  dcode    dname        lat         lng  
0   เปิดเวลา 09:00-21

In [4]:
# Department Clean
dep_cols = ['name', 'address', 'tel', 'time', 'lat', 'lng']
df_department = df_department[dep_cols].copy()

df_department = df_department.dropna(subset=["lat", "lng"])
df_department = df_department.drop_duplicates(subset=["name", "address", "lat", "lng"])

df_department["district"] = df_department["address"].apply(extract_district)

# rename column
df_department = df_department.rename(columns={
    "name": "department_name"
})

df_department = df_department.dropna(how="any").reset_index(drop=True)

df_department


,department_name,address,tel,time,lat,lng,district
0,บิ๊กซี สาขาสุขาภิบาล 3 สาขา 2 (เอ็กซ์ตร้า),643 ถนน รามคำแหง แขวงหัวหมาก เขตบางกะปิ กรุงเท...,0 2735 3062,เปิดเวลา 09:00-21:00น.,13.770855,100.658397,บางกะปิ
1,โลตัส จรัญสนิทวงศ์,244 ซอย จรัญสนิทวงศ์ 79 แขวงบางพลัด เขตบางพลัด...,0 2434 7575,เปิดเวลา 08:00-22:00น.,13.787928,100.502385,บางพลัด
2,โลตัส แจ้งวัฒนะ,300 ม.1 ถ.แจ้งวัฒนะ แขวงทุ่งสองห้อง เขตหลักสี่...,0 2990 7580,เปิดเวลา 08:00-22:00น.,13.895829,100.556153,หลักสี่
3,โลตัส ซีคอนสแควร์,904/2 ม.6 ถ.ศรีนครินทร์ แขวงหนองบอน เขตประเวศ ...,0 2721 9118,เปิดเวลา 09:00-23:00น.,13.692540,100.648029,ประเวศ
4,โลตัส บางแค,266 ถ. เพชรเกษม แขวงบางแคเหนือ เขตบางแค กทม. 1...,0 2804 4837,เปิดเวลา 08:00-23:00 น.,13.712653,100.418968,บางแค
...,...,...,...,...,...,...,...
145,โลตัส พัฒนาการ,2279 ถ.พัฒนาการ เขตสวนหลวง แขวงสวนหลวง กทม.10250,0 2722 8100,เปิดเวลา 08:00-22:00น.,13.737418,100.635712,สวนหลวง
146,โลตัส หนองจอก,46 ถ.เชื่อมสัมพันธ์ แขวงกระทุ่มราย เขตหนองจอก ...,0 2988 3383,เปิดเวลา 08:00-21:00น.,13.851408,100.859707,หนองจอก
147,โลตัส วังหิน,"555,557 แขวงลาดพร้าว เขตลาดพร้าว กทม. 10230",0 2942 3762,เปิดเวลา 08:00-22:00น.,13.823920,100.590352,ลาดพร้าว
148,โลตัส อ่อนนุช 80,172/5 ถ.อ่อนนุช แขวงประเวศ เขตประเวศ กทม.10250,0 2720 3714,เปิดเวลา 07:00-22:00น.,13.718148,100.670693,ประเวศ


In [5]:
# Community
url_community = "https://cpudgiapp.bangkok.go.th/arcgis/rest/services/Community/Service_Community_Public/MapServer/0/query"
params = {
    "where": "1=1",
    "outFields": "*",
    "f": "json",
    "returnGeometry": "true"
}

data_community = requests.get(url_community, params=params).json()
df_community = pd.DataFrame([f["attributes"] for f in data_community["features"]])
print("Columns:", df_community.columns)
print("Examples:")
df_community.head()

Columns: Index(['OBJECTID', 'CMT_TYPE', 'DNAME', 'SNAME', 'NHOUSE', 'RAI', 'NGAN', 'WA',
       'HOUSEHOLD', 'MALE', 'FEMALE', 'ADDRESS', 'NORTH_BND', 'SOUTH_BND',
       'EAST_BND', 'WEST_BND', 'REMARK', 'LON', 'LAT',
       'COMMU_ADMIN_COMMU_BND_FINAL_CM', 'COMMU_ADMIN_COMMU_BND_FINAL__1',
       'COMMU_ADMIN_COMMU_BND_FINAL_DC', 'CMT_ID', 'CMT_NAME', 'STATUS',
       'ORIG_FID', 'CREATED_USER', 'CREATED_DATE', 'LAST_EDITED_USER',
       'LAST_EDITED_DATE', 'ESTABLISHED'],
      dtype='object')
Examples:


,OBJECTID,CMT_TYPE,DNAME,SNAME,NHOUSE,RAI,NGAN,WA,HOUSEHOLD,MALE,...,COMMU_ADMIN_COMMU_BND_FINAL_DC,CMT_ID,CMT_NAME,STATUS,ORIG_FID,CREATED_USER,CREATED_DATE,LAST_EDITED_USER,LAST_EDITED_DATE,ESTABLISHED
0,52,ชุมชนชานเมือง,หนองจอก,กระทุ่มราย,128.0,400.0,0.0,0.0,128.0,590.0,...,1003,100301012.0,เกาะกลางพัฒนา,1,101,None,None,TRAINEE03,1762927903000,1091059200000
1,53,ชุมชนเมือง,หนองจอก,กระทุ่มราย,104.0,20.0,0.0,0.0,104.0,134.0,...,1003,100301013.0,หมู่บ้านจามจุรี,1,102,None,None,TRAINEE03,1763349621000,1091059200000
2,54,ชุมชนชานเมือง,หนองจอก,กระทุ่มราย,111.0,300.0,0.0,0.0,111.0,270.0,...,1003,100301014.0,ป.เจริญมิตรพัฒนา,1,103,None,None,TRAINEE03,1763015870000,1135296000000
3,55,ชุมชนชานเมือง,หนองจอก,กระทุ่มราย,142.0,1200.0,0.0,0.0,142.0,326.0,...,1003,100301015.0,ลำสลิดทองพัฒนา,1,104,None,None,TRAINEE03,1763092567000,1135296000000
4,56,ชุมชนเมือง,หนองจอก,กระทุ่มราย,262.0,20.0,0.0,0.0,262.0,730.0,...,1003,100301016.0,นันทวันเซ็นต์ 1,1,105,None,None,TRAINEE03,1763004579000,1200873600000


In [6]:
# Community Clean
commu_cols = ["CMT_NAME", "CMT_TYPE", "DNAME", "LON", "LAT",
              "RAI", "NGAN", "WA", "NHOUSE", "HOUSEHOLD"]

df_community = df_community[commu_cols].copy()

df_community["household"] = df_community.apply(
    lambda r: r["NHOUSE"] if r["NHOUSE"] == r["HOUSEHOLD"] else r["HOUSEHOLD"],
    axis=1
)

df_community = df_community.drop(columns=["NHOUSE", "HOUSEHOLD"])
df_community = df_community.rename(columns={
    "CMT_NAME": "community_name",
    "CMT_TYPE": "community_type",
    "DNAME": "district",
    "LON": "lng",
    "LAT": "lat",
    "RAI": "rai",
    "NGAN": "ngan",
    "WA": "wa",
})

df_community = df_community.dropna(subset=["lng", "lat"])
df_community["district"] = df_community["district"].astype(str).apply(extract_district)
df_community = df_community.drop_duplicates(subset=["community_name", "district", "lat", "lng"])

df_community = df_community.dropna(how="any").reset_index(drop=True)
df_community


,community_name,community_type,district,lng,lat,rai,ngan,wa,household
0,เกาะกลางพัฒนา,ชุมชนชานเมือง,หนองจอก,100.903907,13.845261,400.0,0.0,0.0,128.0
1,หมู่บ้านจามจุรี,ชุมชนเมือง,หนองจอก,100.883142,13.848948,20.0,0.0,0.0,104.0
2,ป.เจริญมิตรพัฒนา,ชุมชนชานเมือง,หนองจอก,100.924220,13.822292,300.0,0.0,0.0,111.0
3,ลำสลิดทองพัฒนา,ชุมชนชานเมือง,หนองจอก,100.873535,13.837842,1200.0,0.0,0.0,142.0
4,นันทวันเซ็นต์ 1,ชุมชนเมือง,หนองจอก,100.882769,13.844452,20.0,0.0,0.0,262.0
...,...,...,...,...,...,...,...,...,...
995,อยู่เจริญ-บุญมา,ชุมชนเมือง,ดอนเมือง,100.596589,13.944447,30.0,0.0,0.0,583.0
996,ประชาอุทิศม่วงมณีร่วมใจ,ชุมชนเมือง,ดอนเมือง,100.585089,13.918646,40.0,0.0,0.0,302.0
997,แม่ทองก้อน,ชุมชนเมือง,ดอนเมือง,100.594263,13.922006,3.0,0.0,0.0,140.0
998,ศรีดอนเมือง,ชุมชนเมือง,ดอนเมือง,100.594386,13.923430,195.0,0.0,0.0,535.0


In [7]:
# School
url_school = "https://bmagis.bangkok.go.th/arcgis/rest/services/riskbkk/RISK_ADMIN_bma_school/FeatureServer/0/query"
data_school = requests.get(url_school, params=params).json()
df_school = pd.DataFrame([f["attributes"] for f in data_school["features"]])
print("Columns:", df_school.columns)
print("Examples:")
df_school.head()

Columns: Index(['OBJECTID', 'TYPE', 'NAME', 'DCODE', 'ADDRESS', 'X', 'Y', 'GLOBALID',
       'CREATED_USER', 'CREATED_DATE', 'LAST_EDITED_USER', 'LAST_EDITED_DATE'],
      dtype='object')
Examples:


,OBJECTID,TYPE,NAME,DCODE,ADDRESS,X,Y,GLOBALID,CREATED_USER,CREATED_DATE,LAST_EDITED_USER,LAST_EDITED_DATE
0,1,1,โรงเรียนวัดใหม่อมตรส,1001,132 ถ.สามเสน แขวงพานถม เขตพระนคร,662324.254,1522243.929,{6C7A8514-38AE-4D88-B6AE-A7A1B1105A2A},None,NaN,None,NaN
1,2,3,โรงเรียนวัดหนองจอก (ภักดีนรเศรษฐ),1003,18 หมู่ 2 ถ.เลียบวารี แขวงกระทุ่มราย เขตหนองจอก,701051.281,1532706.770,{4FEDB52D-8FDB-4CC4-A706-4B530333C885},None,NaN,None,NaN
2,3,2,โรงเรียนสุเหร่าหะยีมินา,1003,หมู่ 4 ถ.สังฆสันติสุข แขวงกระทุ่มราย เขตหนองจอก,702036.597,1532511.740,{54283143-C638-4E8B-ACBA-E4BF244F928C},None,NaN,None,NaN
3,4,1,โรงเรียนสุเหร่าบ้านเกาะ,1003,14 หมู่ 1 แขวงหนองจอก เขตหนองจอก,703531.964,1534628.348,{9E47C92F-3EBB-4F01-A680-5F05E904B64D},None,NaN,None,NaN
4,5,1,โรงเรียนลำบุหรี่พวง,1003,หมู่ 14 ลำบุหรี่พวง แขวงคลองสิบ เขตหนองจอก,699856.600,1536112.920,{03704461-A0B1-4EDB-B8EB-90F49E409BEB},None,NaN,None,NaN


In [8]:
# School Clean
school_cols = ["TYPE", "NAME", "DCODE", "ADDRESS", "X", "Y"]
df_school = df_school[school_cols].copy()

df_school["X"] = pd.to_numeric(df_school["X"], errors="coerce")
df_school["Y"] = pd.to_numeric(df_school["Y"], errors="coerce")
df_school = df_school.dropna(subset=["X", "Y"])

df_school = gpd.GeoDataFrame(
    df_school,
    geometry=gpd.points_from_xy(df_school["X"], df_school["Y"]),
    crs="EPSG:32647"
).to_crs(epsg=4326)

df_school["lat"] = df_school.geometry.y
df_school["lng"] = df_school.geometry.x

size_map = {
    "1": "โรงเรียนขนาดเล็ก",
    "2": "โรงเรียนขนาดกลาง",
    "3": "โรงเรียนขนาดใหญ่"
}

df_school["school_size"] = df_school["TYPE"].astype(str).map(size_map).fillna("ไม่ทราบ")
df_school["district"] = df_school["ADDRESS"].apply(extract_district)

df_school = df_school.rename(columns={
    "NAME": "school_name",
    "ADDRESS": "address",
})

df_school = df_school.drop_duplicates(subset=["school_name", "lat", "lng"])
df_school = df_school.dropna(how="any").reset_index(drop=True)

df_school = df_school[[
    "school_name", "school_size", "district", "address", "lat", "lng"
]]

df_school


,school_name,school_size,district,address,lat,lng
0,โรงเรียนวัดใหม่อมตรส,โรงเรียนขนาดเล็ก,พระนคร,132 ถ.สามเสน แขวงพานถม เขตพระนคร,13.765009,100.501464
1,โรงเรียนวัดหนองจอก (ภักดีนรเศรษฐ),โรงเรียนขนาดใหญ่,หนองจอก,18 หมู่ 2 ถ.เลียบวารี แขวงกระทุ่มราย เขตหนองจอก,13.857122,100.860310
2,โรงเรียนสุเหร่าหะยีมินา,โรงเรียนขนาดกลาง,หนองจอก,หมู่ 4 ถ.สังฆสันติสุข แขวงกระทุ่มราย เขตหนองจอก,13.855291,100.869409
3,โรงเรียนสุเหร่าบ้านเกาะ,โรงเรียนขนาดเล็ก,หนองจอก,14 หมู่ 1 แขวงหนองจอก เขตหนองจอก,13.874313,100.883395
4,โรงเรียนลำบุหรี่พวง,โรงเรียนขนาดเล็ก,หนองจอก,หมู่ 14 ลำบุหรี่พวง แขวงคลองสิบ เขตหนองจอก,13.887989,100.849503
...,...,...,...,...,...,...
431,โรงเรียนปลูกจิต,โรงเรียนขนาดใหญ่,ปทุมวัน,99 ซ.ปลูกจิตต์ แขวงลุมพินี เขตปทุมวัน,13.728496,100.549965
432,โรงเรียนวัดจักรวรรดิ์,โรงเรียนขนาดเล็ก,สัมพันธวงศ์,225 ถ.จักรวรรดิ แขวงจักรวรรดิ แขวงสัมพันธวงศ์ ...,13.742665,100.503675
433,โรงเรียนหมู่บ้านเกาะโพธิ์,โรงเรียนขนาดเล็ก,บางขุนเทียน,22/3 หมู่ 10 แขวงแสมดำ เขตบางขุนเทียน,13.571034,100.389946
434,โรงเรียนมัธยมประชานิเวศน์,โรงเรียนขนาดใหญ่,จตุจักร,ถ.เทศบาลรังรักษ์เหนือ แขวงลาดยาว เขตจตุจักร,13.841714,100.548284


In [9]:
# Hospital
url_hospital = "https://bmagis.bangkok.go.th/arcgis/rest/services/riskbkk/RISK_ADMIN_Hospital/FeatureServer/0/query"
data_hospital = requests.get(url_hospital, params=params).json()
df_hospital = pd.DataFrame([f["attributes"] for f in data_hospital["features"]])
print("Columns:", df_hospital.columns)
print("Examples:")
df_hospital.head()


Columns: Index(['OBJECTID', 'NAME', 'DCODE', 'ADDRESS', 'TEL', 'NUM_BED', 'X', 'Y'], dtype='object')
Examples:


,OBJECTID,NAME,DCODE,ADDRESS,TEL,NUM_BED,X,Y
0,1,โรงพยาบาลภูมิพลอดุลยเดช,1042,171 ถ.พหลโยธิน แขวงคลองถนน เขตสายไหม กทม.10220,0 - 2534-7000,700,674848.512,1538263.349
1,2,โรงพยาบาลและสถาบันประสาทวิทยา,1037,312 ถ. ราชวิถี แขวงทุ่งพญาไท เขตราชเทวี กทม.,0 - 2246 - 9254,350,665118.241,1522693.716
2,3,โรงพยาบาลพระมงกุฏเกล้า,1037,315 ถ.ราชวิถี ทุ่งพญาไท เขตราชเทวี กทม.10400,0 - 2246 - 1400-28,1236,665789.636,1522522.013
3,4,โรงพยาบาลราชวิถี,1037,2 ถ.พญาไท แขวงทุ่งพญาไท เขตราชเทวี กทม.10400,0 - 2248 - 3213-4,1235,666049.879,1522231.596
4,5,โรงพยาบาลสงฆ์,1037,445 ถ.ศรีอยุธยา แขวงทุ่งพญาไท เขตราชเทวี กทม.1...,0 - 2247 - 1655,428,665042.941,1521929.200


In [10]:
# Hospital Clean
hospital_cols = ["NAME", "ADDRESS", "TEL", "X", "Y"]
df_hospital = df_hospital[hospital_cols].copy()

df_hospital["X"] = pd.to_numeric(df_hospital["X"], errors="coerce")
df_hospital["Y"] = pd.to_numeric(df_hospital["Y"], errors="coerce")
df_hospital = df_hospital.dropna(subset=["X", "Y"])

df_hospital = gpd.GeoDataFrame(
    df_hospital,
    geometry=gpd.points_from_xy(df_hospital["X"], df_hospital["Y"]),
    crs="EPSG:32647"
).to_crs(epsg=4326)

df_hospital["lat"] = df_hospital.geometry.y
df_hospital["lng"] = df_hospital.geometry.x
df_hospital["district"] = df_hospital["ADDRESS"].apply(extract_district)

df_hospital = df_hospital.rename(columns={
    "NAME": "hospital_name",
    "ADDRESS": "address",
    "TEL": "tel"
})

df_hospital = df_hospital.drop_duplicates(subset=["hospital_name", "lat", "lng"])
df_hospital = df_hospital.dropna(how="any").reset_index(drop=True)

df_hospital = df_hospital[[
    "hospital_name", "address", "district", "tel", "lat", "lng"
]]

df_hospital


,hospital_name,address,district,tel,lat,lng
0,โรงพยาบาลภูมิพลอดุลยเดช,171 ถ.พหลโยธิน แขวงคลองถนน เขตสายไหม กทม.10220,สายไหม,0 - 2534-7000,13.909068,100.618280
1,โรงพยาบาลและสถาบันประสาทวิทยา,312 ถ. ราชวิถี แขวงทุ่งพญาไท เขตราชเทวี กทม.,ราชเทวี,0 - 2246 - 9254,13.768916,100.527328
2,โรงพยาบาลพระมงกุฏเกล้า,315 ถ.ราชวิถี ทุ่งพญาไท เขตราชเทวี กทม.10400,ราชเทวี,0 - 2246 - 1400-28,13.767326,100.533526
3,โรงพยาบาลราชวิถี,2 ถ.พญาไท แขวงทุ่งพญาไท เขตราชเทวี กทม.10400,ราชเทวี,0 - 2248 - 3213-4,13.764686,100.535916
4,โรงพยาบาลสงฆ์,445 ถ.ศรีอยุธยา แขวงทุ่งพญาไท เขตราชเทวี กทม.1...,ราชเทวี,0 - 2247 - 1655,13.762010,100.526587
...,...,...,...,...,...,...
125,โรงพยาบาลวัฒโนสถ,85ศูนย์วิจัย เพชรบุรีตัดใหม่ บางกะปิ ห้วยขวาง...,ห้วยขวาง,02310-3000,13.756059,100.581966
126,สถานพยาบาลเวชกรรม กรุงเทพอินเตอร์เนชั่นแนล,ศูนย์วิจัย 7 เพชรบุรีตัดใหม่ บางกะปิ ห้วยขวาง ...,ห้วยขวาง,023103227,13.744905,100.584813
127,โรงพยาบาลลาดพร้าว,2699 ลาดพร้าว วังทองหลาง แขวงวังทองหลาง กรุงเท...,ลาดพร้าว,02-530-2556-69,13.778555,100.623888
128,โรงพยาบาลการแพทย์วิชัยยุทธ,53ซอยเศรษฐศิริ สามเสนใน แขวงพญาไท กรุงเทพฯ 10400,พญาไท,02-265-7777,13.783266,100.533685


In [11]:
folder = "data"

# Export Department
# [department_name, address, tel, time, lat, lng, district]
print("Department columns:", list(df_department.columns))
df_department.to_csv(os.path.join(folder, "department_clean.csv"), index=False)
print("Saved: department_clean.csv")


# Export Community
# [community_name, community_type, district, lng, lat, rai, ngan, wa, household]
print("Community columns:", list(df_community.columns))
df_community.to_csv(os.path.join(folder, "department_clean.csv"), index=False)
print("Saved: community_clean.csv")


# Export School
# [school_name, school_size, district, address, lat, lng]
print("School columns:", list(df_school.columns))
df_school.to_csv(os.path.join(folder, "school_clean.csv"), index=False)
print("Saved: school_clean.csv")


# Export Hospital
# [hospital_name, address, district, tel, lat, lng]
print("Hospital columns:", list(df_hospital.columns))
df_hospital.to_csv(os.path.join(folder, "hospital_clean.csv"), index=False)
print("Saved: hospital_clean.csv")


Department columns: ['department_name', 'address', 'tel', 'time', 'lat', 'lng', 'district']
Saved: department_clean.csv
Community columns: ['community_name', 'community_type', 'district', 'lng', 'lat', 'rai', 'ngan', 'wa', 'household']
Saved: community_clean.csv
School columns: ['school_name', 'school_size', 'district', 'address', 'lat', 'lng']
Saved: school_clean.csv
Hospital columns: ['hospital_name', 'address', 'district', 'tel', 'lat', 'lng']
Saved: hospital_clean.csv
